In [1]:
import os
from pathlib import Path

from logs.wandblogger import WandBLogger2D
from training.trainer import MRTrainer
from datasets.signals import ImageSignal#, make_mask
from networks.mrnet import MRFactory
from datasets.pyramids import create_MR_structure
import yaml
from yaml.loader import SafeLoader
import os

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "train-wb.ipynb"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
BASE_DIR = Path('.').absolute().parents[0]
IMAGE_PATH = BASE_DIR.joinpath('img')
MODEL_PATH = BASE_DIR.joinpath('models')

In [3]:
project_name = "dev-sandbox"
#-- hyperparameters in configs --#
config_file = '../configs/config_base_m_net.yml'
with open(config_file) as f:
    hyper = yaml.load(f, Loader=SafeLoader)
    if isinstance(hyper['batch_size'], str):
        hyper['batch_size'] = eval(hyper['batch_size'])
    print(hyper)
imgpath = os.path.join(IMAGE_PATH, hyper['image_name'])
maskpath = None
# maskpath = "/Users/hallpaz/Workspace/impa/mrimg/img/synthetic/mask_inverted.png" #make_mask(imgpath, hyper['mask_color'])

{'model': 'M', 'in_features': 2, 'hidden_layers': 1, 'hidden_features': [8, 8, 16, 32, 64, 128, 192], 'bias': True, 'max_stages': 7, 'period': 2, 'domain': [-1, 1], 'omega_0': [1, 2, 4, 6, 8, 16, 32], 'hidden_omega_0': [30, 30, 30, 30, 30, 30, 30], 'superposition_w0': False, 'sampling_scheme': 'regular', 'decimation': True, 'filter': 'gauss', 'attributes': ['d0'], 'loss_function': 'mse', 'opt_method': 'Adam', 'lr': 0.0005, 'loss_tol': 1e-14, 'diff_tol': 1e-11, 'max_epochs_per_stage': [601, 401, 401, 401, 401, 401, 401], 'batch_size': 16384, 'image_name': 'hallpaz.jpg', 'width': 512, 'height': 512, 'channels': 1, 'device': 'cuda', 'eval_device': 'cpu', 'save_format': 'general', 'visualize_grad': True, 'extrapolate': [-2, 2]}


In [4]:
base_signal = ImageSignal.init_fromfile(
                    imgpath,
                    domain=hyper['domain'],
                    channels=hyper['channels'],
                    sampling_scheme=hyper['sampling_scheme'],
                    width=hyper['width'], height=hyper['height'],
                    attributes=hyper['attributes'],
                    batch_size=hyper['batch_size'])

train_dataloader = create_MR_structure(base_signal, hyper['max_stages'], 
                                       hyper['filter'], hyper['decimation'])
test_dataloader = create_MR_structure(base_signal, hyper['max_stages'], 
                                      hyper['filter'])

In [5]:
hyper['device']

'cuda'

In [6]:
mrmodel2 = MRFactory.from_dict(hyper).to(hyper['device'])

In [7]:
img_name = os.path.basename(hyper['image_name'])
wandblogger = WandBLogger2D(project_name,
                            f"{hyper['model']}{hyper['filter'][0].upper()}{img_name[0:5]}",
                            hyper,
                            BASE_DIR,
                            visualize_gt_grads=hyper.get('visualize_grad', False))
mrmodel = MRFactory.from_dict(hyper)
print("Model: ", type(mrmodel))
mrtrainer = MRTrainer.init_from_dict(mrmodel, train_dataloader, test_dataloader, wandblogger, hyper)
mrtrainer.train(hyper['device'])

Model:  <class 'networks.mrnet.MNet'>


wandb: Currently logged in as: hallpaz (siren-song). Use `wandb login --relogin` to force relogin


No gradients in sampler and visualization is True. Set visualize_grad to False
[Logger] All inference done in 0.9902336597442627s on cpu


D0 loss,█▇▆▅▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
D0 loss,0.00941


No gradients in sampler and visualization is True. Set visualize_grad to False
[Logger] All inference done in 0.7921226024627686s on cpu


...

No gradients in sampler and visualization is True. Set visualize_grad to False
[Logger] All inference done in 4.174679756164551s on cpu


D0 loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
D0 loss,0.00049


No gradients in sampler and visualization is True. Set visualize_grad to False
[Logger] All inference done in 10.307757377624512s on cpu
File  MGhallp_7-7_w32F_hf192_MEp401_hl1_512px.pth


D0 loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
D0 loss,0.00055


Total model parameters =  96512
Training finished after 3007 epochs
